# Classificação Binária da presença de fogo em imagens com Yolov11

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
#Importe das bibliotecas necessárias
import os
import random
import shutil
from pathlib import Path
import yaml
from ultralytics import YOLO
import kagglehub

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Etapa 1: Baixar o dataset
path = kagglehub.dataset_download("phylake1337/fire-dataset")
original_dataset_path = Path(path) / "fire_dataset"  # Corrige o caminho real
output_dataset_path = Path("dataset")
yaml_path = output_dataset_path / "data.yaml"

100%|██████████| 387M/387M [00:02<00:00, 177MB/s]

Extracting files...


In [ ]:
# Mapear nomes das pastas reais para rótulos
dataset_dirs = {
    "fire": "fire_images",
    "non_fire": "non_fire_images"
}
classes = list(dataset_dirs.keys())

# Parâmetro de divisão
train_ratio = 0.8

# Etapa 2: Criar estrutura de pastas
def create_folders():
    for split in ["train", "val", "test"]:
        for cls in classes:
            (output_dataset_path / split / cls).mkdir(parents=True, exist_ok=True)


In [ ]:
# Etapa 3: Copiar e dividir imagens
def split_dataset(source_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "As proporções devem somar 1.0"

    # Mapeia nomes das pastas para classes no YAML
    class_map = {
        "fire_images": "fire",
        "non_fire_images": "non_fire"
    }

    for folder_name, class_name in class_map.items():
        class_path = Path(source_dir) / folder_name
        images = list(class_path.glob("*.png")) + list(class_path.glob("*.jpg"))
        random.shuffle(images)

        n_total = len(images)
        n_train = int(train_ratio * n_total)
        n_val = int(val_ratio * n_total)
        n_test = n_total - n_train - n_val

        splits = {
            "train": images[:n_train],
            "val": images[n_train:n_train + n_val],
            "test": images[n_train + n_val:]
        }

        for split, split_images in splits.items():
            split_dir = Path(output_dir) / split / class_name
            split_dir.mkdir(parents=True, exist_ok=True)
            for img_path in split_images:
                shutil.copy(img_path, split_dir)

        print(f"\nClasse '{class_name}':")
        print(f"  Treino: {n_train} imagens")
        print(f"  Validação: {n_val} imagens")
        print(f"  Teste: {n_test} imagens")


In [ ]:
# Etapa 4: Criar data.yaml
def create_yaml():
    data = {
        "train": str((output_dataset_path / "train").resolve()),
        "val": str((output_dataset_path / "val").resolve()),
        "test": str((output_dataset_path / "test").resolve()),
        "nc": len(classes),
        "names": classes
    }
    with open(yaml_path, "w") as f:
        yaml.dump(data, f)

In [ ]:
# Etapa 5: Verificar distribuição
def show_distribution():
    print("Distribuição de imagens:")
    for split in ["train", "val"]:
        print(f"\n{split.upper()}")
        for cls in classes:
            count = len(list((output_dataset_path / split / cls).glob("*.*")))
            print(f"  {cls}: {count} imagens")

# Etapa 6: Treinar o modelo
def train_model():

    print("\nTreinando o modelo YOLOv11...")
    model = YOLO("yolo11n-cls.pt")
    results = model.train(data="/content/dataset", epochs=25, imgsz=640, optimizer="Adam")
    print("\nTreinamento concluído!")

In [ ]:
# Execução
if __name__ == "__main__":
    create_folders()
    split_dataset(source_dir=original_dataset_path, output_dir=output_dataset_path)
    create_yaml()
    show_distribution()
    train_model()



Classe 'fire':
  Treino: 528 imagens
  Validação: 113 imagens
  Teste: 114 imagens

Classe 'non_fire':
  Treino: 170 imagens
  Validação: 36 imagens
  Teste: 38 imagens
Distribuição de imagens:

TRAIN
  fire: 528 imagens
  non_fire: 170 imagens

VAL
  fire: 113 imagens
  non_fire: 36 imagens

Treinando o modelo YOLOv11...


100%|██████████| 5.52M/5.52M [00:00<00:00, 68.4MB/s]


Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, p

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.8MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1890.0±1025.6 MB/s, size: 91.6 KB)


...

      25/25      2.38G     0.1063          9        640: 100%|██████████| 44/44 [00:43<00:00,  1.01it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:02<00:00,  2.26it/s]

                   all      0.946          1



25 epochs completed in 0.319 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.2MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: /content/dataset/train... found 698 images in 2 classes ✅ 
val: /content/dataset/val... found 149 images in 2 classes ✅ 
test: /content/dataset/test... found 152 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:12<00:00,  2.56s/it]


                   all      0.987          1
Speed: 0.5ms preprocess, 1.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train

Treinamento concluído!


#Avaliação no conjunto de teste

In [ ]:
model = YOLO("runs/classify/train/weights/best.pt")
metrics = model.val(split="test", data=str(yaml_path.resolve()))

Ultralytics 8.3.166 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-cls summary (fused): 47 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
test: Fast image access ✅ (ping: 0.0±0.0 ms, read: 951.5±984.8 MB/s, size: 95.8 KB)


test: Scanning /content/dataset/test... 152 images, 0 corrupt: 100%|██████████| 152/152 [00:00<00:00, 2417.90it/s]

test: New cache created: /content/dataset/test.cache



               classes   top1_acc   top5_acc: 100%|██████████| 10/10 [00:18<00:00,  1.86s/it]


                   all      0.993          1
Speed: 2.1ms preprocess, 5.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val
